In [ ]:
import numpy as np
import argparse
import sys
import pandas as pd
import shutil
import tempfile
import tensorflow as tf
from tensorflow import estimator as tf_estimator


To connect to tour dags hub account replace username and project name in below code ,
Add your Dagshub key when propmted

In [ ]:
!pip install mlflow --quiet

import mlflow
import os
from getpass import getpass

os.environ['MLFLOW_TRACKING_USERNAME'] = 'vedantd1998'
os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('enter access token')
os.environ['MLFLOW_TRACKING_PROJECTNAME'] = 'ML'

mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME'] + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')


     |████████████████████████████████| 16.5 MB 186 kB/s 
     |████████████████████████████████| 62 kB 539 kB/s 
     |████████████████████████████████| 210 kB 53.7 MB/s 
     |████████████████████████████████| 596 kB 32.0 MB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 146 kB 61.9 MB/s 
     |████████████████████████████████| 181 kB 44.7 MB/s 
     |████████████████████████████████| 54 kB 2.0 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
enter access token··········


In [ ]:
import mlflow
import keras

In [ ]:
from imblearn.over_sampling import ADASYN

train_df = pd.read_csv(r'/content/test_data.csv',index_col=0)
X=train_df.drop('credit_card_default',axis=1)
y=train_df['credit_card_default']


smote = ADASYN(n_neighbors=5)
X_train, y_train = smote.fit_resample(X, y)




test_df = pd.read_csv(r'/content/test_data.csv',index_col=0)
X_test=test_df.drop('credit_card_default',axis=1)
y_test=test_df['credit_card_default']


In [ ]:

with mlflow.start_run(run_name="MLflow on Colab"):
  

  tf.random.set_seed(2022)
  model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(13, activation='relu',input_shape=(X_train.shape[1], )),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(8, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(1, activation='sigmoid')  
  ])



  model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['Precision'])

  from tensorflow.keras.callbacks import EarlyStopping

  monitor = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=20, verbose=2, mode='auto',
          restore_best_weights=True)
  history2 = model.fit(X_train,y_train,validation_data=(X_test,y_test),callbacks=[monitor],verbose=2,epochs=500)

  y_pred_prob = model.predict(X_test)
  y_pred = np.where(y_pred_prob>=0.5,1,0)

  from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss, precision_score, recall_score


  mlflow.log_metric("accuracy", accuracy_score(y_test,y_pred))
  mlflow.log_metric("f1", f1_score(y_test,y_pred))
  mlflow.log_metric("precsion", precision_score(y_test,y_pred))
  mlflow.log_metric("recall", recall_score(y_test,y_pred))





Epoch 1/500
786/786 - 4s - loss: 0.4260 - precision: 0.7842 - val_loss: 0.1935 - val_precision: 0.4665 - 4s/epoch - 5ms/step
Epoch 2/500
786/786 - 2s - loss: 0.2055 - precision: 0.9065 - val_loss: 0.1537 - val_precision: 0.5411 - 2s/epoch - 3ms/step
Epoch 3/500
786/786 - 2s - loss: 0.1528 - precision: 0.9283 - val_loss: 0.1504 - val_precision: 0.5757 - 2s/epoch - 3ms/step
Epoch 4/500
786/786 - 2s - loss: 0.1370 - precision: 0.9336 - val_loss: 0.1366 - val_precision: 0.5960 - 2s/epoch - 3ms/step
Epoch 5/500
786/786 - 2s - loss: 0.1268 - precision: 0.9381 - val_loss: 0.1266 - val_precision: 0.6174 - 2s/epoch - 3ms/step
Epoch 6/500
786/786 - 2s - loss: 0.1217 - precision: 0.9403 - val_loss: 0.1269 - val_precision: 0.6204 - 2s/epoch - 3ms/step
Epoch 7/500
786/786 - 2s - loss: 0.1148 - precision: 0.9422 - val_loss: 0.1257 - val_precision: 0.6294 - 2s/epoch - 3ms/step
Epoch 8/500
786/786 - 2s - loss: 0.1138 - precision: 0.9428 - val_loss: 0.1201 - val_precision: 0.6322 - 2s/epoch - 3ms/step


In [ ]:
import IPython
display(IPython.display.IFrame("https://dagshub.com/"+ os.environ['MLFLOW_TRACKING_USERNAME'] 
                        + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + "/experiments/#/",'100%',600))